In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from io import StringIO

warnings.filterwarnings('ignore')

In [ ]:
def load_terrorism_file(filename: str) -> pd.DataFrame: #has country,target type
    """Load a file efficiently, retaining only the most useful columns & rows.
     Uses Pandas read_csv() with its compression='infer' option.
     param df: reads the file
    """
    
    csv_columns = ['eventid','iyear','imonth','iday','approxdate','extended','resolution','country','country_txt','region','region_txt','provstate','city','latitude','longitude','specificity','vicinity','location','summary','crit1','crit2','crit3','doubtterr','alternative','alternative_txt','multiple','success','suicide','attacktype1','attacktype1_txt','attacktype2','attacktype2_txt','attacktype3','attacktype3_txt','targtype1','targtype1_txt','targsubtype1','targsubtype1_txt','corp1','target1','natlty1','natlty1_txt','targtype2','targtype2_txt','targsubtype2','targsubtype2_txt','corp2','target2','natlty2','natlty2_txt','targtype3','targtype3_txt','targsubtype3','targsubtype3_txt','corp3','target3','natlty3','natlty3_txt','gname','gsubname','gname2','gsubname2','gname3','gsubname3','motive','guncertain1','guncertain2','guncertain3','individual','nperps','nperpcap','claimed','claimmode','claimmode_txt','claim2','claimmode2','claimmode2_txt','claim3','claimmode3','claimmode3_txt','compclaim','weaptype1','weaptype1_txt','weapsubtype1','weapsubtype1_txt','weaptype2','weaptype2_txt','weapsubtype2','weapsubtype2_txt','weaptype3','weaptype3_txt','weapsubtype3','weapsubtype3_txt','weaptype4','weaptype4_txt','weapsubtype4','weapsubtype4_txt','weapdetail','nkill','nkillus','nkillter','nwound','nwoundus','nwoundte','property','propextent','propextent_txt','propvalue','propcomment','ishostkid','nhostkid','nhostkidus','nhours','ndays','divert','kidhijcountry','ransom','ransomamt','ransomamtus','ransompaid','ransompaidus','ransomnote','hostkidoutcome','hostkidoutcome_txt','nreleased','addnotes','scite1','scite2','scite3','dbsource','INT_LOG','INT_IDEO','INT_MISC','INT_ANY','related']
    
    columns_wanted = ['iyear','country_txt','attacktype1_txt','targtype1_txt']
    
    df = pd.read_csv('terrorism.csv',
                        compression='infer',
                        names=csv_columns,
                        usecols=columns_wanted,)
    df['iyear'] = pd.to_numeric(df['iyear'], errors='coerce')
    
    return df

In [ ]:
def load_perp_file(filename: str) -> pd.DataFrame: 
    """Load a file efficiently, retaining only the most useful columns & rows.
     Uses Pandas read_csv() with its compression='infer' option.
     param df: reads the file
    """

    csv_columns = ['person_id','first_name','last_name','full_name','headshot','headshot_credit','gender','age','inv_informant','inv_public_tip','inv_community_or_family_tip','marital_status','terror_plot','terror_plot_2','plot_id','citizenship_status','charged_or_deceased','year_charged_or_deceased','date_charged','state_charged','state_charged_2','last_residency_state','last_residency_country','char_awlaki','char_contact_with_foreign_militant','char_overseas_military_training','char_us_military_experience','char_online_radicalization','targeted_jews_israel','targeted_military_installation']
    
    columns_wanted = ['first_name','last_name','full_name','age','citizenship_status','marital_status']
    
    df = pd.read_csv('perps.csv',
    compression='infer',
    names=csv_columns,
    usecols=columns_wanted,
    )   
    
    df['age'] = pd.to_numeric(df['age'],errors='coerce')

    return df

In [ ]:
#main function
if __name__ == '__main__' :
    
    terrorism_data = load_terrorism_file('terrorism.csv')  # Loading Terrorism data for all countries
    perpetrators_data = load_perp_file('perps.csv')         # age, marital ,cit status
    terrorists_data = pd.read_csv('terrorists.csv')         # country and cit status
    income_data = pd.read_csv('income.csv')                 # has income for first and third world countries
    population_data = pd.read_csv('population_data.csv') 